## 1. Import Library yang Diperlukan

Pada tahap awal, kita perlu mengimpor semua library yang akan digunakan dalam proses data wrangling. Library ini mencakup tools untuk manipulasi data, visualisasi, dan text processing.

In [ ]:
# Library untuk manipulasi dan analisis data
import pandas as pd
import numpy as np

# Library untuk visualisasi data
import matplotlib.pyplot as plt
import seaborn as sns

# Library untuk text processing dan regular expressions
import re

# Library untuk machine learning dan split data
from sklearn.model_selection import train_test_split

# Library untuk deep learning dengan transformers
from transformers import AutoTokenizer
import torch
from torch.utils.data import TensorDataset, DataLoader

# Konfigurasi untuk tampilan yang lebih baik
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
sns.set_style("whitegrid")

print("Semua library berhasil diimpor.")

## 2. Load Dataset

Tahap pertama dalam data wrangling adalah memuat dataset dari sumber yang tersedia. Dataset ini diambil dari GitHub repository dalam format CSV.

In [ ]:
# Memuat dataset dari URL GitHub
try:
    df = pd.read_csv('https://github.com/erlanggadewasakti/Prinsip-Sains-Data/releases/download/prod/sa-psd-dataset.csv')
    print(f"Dataset berhasil dimuat dengan {len(df)} baris data.")
    print(f"Jumlah kolom: {len(df.columns)}")
except FileNotFoundError:
    print("Error: File dataset tidak ditemukan. Pastikan URL sudah benar.")
except Exception as e:
    print(f"Terjadi error saat membaca file: {e}")

## 3. Inspeksi Awal Dataset

Sebelum melakukan cleaning, kita perlu memahami struktur dan karakteristik dataset. Ini termasuk melihat sample data, tipe data, dan informasi statistik dasar.

In [ ]:
# Menampilkan 5 baris pertama dataset
print("=" * 80)
print("SAMPLE DATA (5 baris pertama)")
print("=" * 80)
display(df.head())

print("\n" + "=" * 80)
print("INFORMASI STRUKTUR DATASET")
print("=" * 80)
df.info()

print("\n" + "=" * 80)
print("STATISTIK DESKRIPTIF")
print("=" * 80)
display(df.describe(include='all'))

## 4. Identifikasi dan Penanganan Missing Values

Missing values bisa mempengaruhi kualitas analisis dan model. Kita perlu mengidentifikasi kolom mana yang memiliki nilai kosong dan menentukan strategi penanganannya.

In [ ]:
# Mengidentifikasi missing values
print("=" * 80)
print("ANALISIS MISSING VALUES")
print("=" * 80)

missing_values = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Kolom': missing_values.index,
    'Jumlah Missing': missing_values.values,
    'Persentase (%)': missing_percentage.values
})

print(missing_df[missing_df['Jumlah Missing'] > 0])

if missing_df['Jumlah Missing'].sum() == 0:
    print("\nTidak ditemukan missing values dalam dataset.")
else:
    print(f"\nTotal missing values: {missing_df['Jumlah Missing'].sum()}")

# Visualisasi missing values jika ada
if missing_df['Jumlah Missing'].sum() > 0:
    plt.figure(figsize=(10, 6))
    missing_cols = missing_df[missing_df['Jumlah Missing'] > 0]
    plt.barh(missing_cols['Kolom'], missing_cols['Persentase (%)'])
    plt.xlabel('Persentase Missing Values (%)')
    plt.title('Distribusi Missing Values per Kolom')
    plt.tight_layout()
    plt.show()

In [ ]:
# Menangani missing values jika ada
# Strategi: Drop baris dengan missing values pada kolom penting (input dan output)
# karena kedua kolom ini essential untuk analisis sentimen

df_clean = df.copy()

if df_clean.isnull().sum().sum() > 0:
    # Drop baris yang memiliki missing values pada kolom input atau output
    initial_rows = len(df_clean)
    df_clean = df_clean.dropna(subset=['input', 'output'], how='any')
    dropped_rows = initial_rows - len(df_clean)

    print(f"Jumlah baris yang dihapus karena missing values: {dropped_rows}")
    print(f"Sisa data setelah penanganan missing values: {len(df_clean)} baris")
else:
    print("Tidak ada missing values yang perlu ditangani.")

# Verifikasi tidak ada lagi missing values
print(f"\nVerifikasi - Total missing values setelah cleaning: {df_clean.isnull().sum().sum()}")

## 5. Identifikasi dan Penanganan Data Duplikat

Data duplikat dapat menyebabkan bias dalam analisis dan model. Kita perlu mengidentifikasi dan menghapus baris yang benar-benar duplikat.

In [ ]:
# Mengidentifikasi duplikasi
print("=" * 80)
print("ANALISIS DATA DUPLIKAT")
print("=" * 80)

# Cek duplikasi pada semua kolom
duplicates_all = df_clean.duplicated().sum()
print(f"Jumlah baris duplikat (semua kolom): {duplicates_all}")

# Cek duplikasi spesifik pada kolom 'input' karena ini adalah text utama
duplicates_input = df_clean.duplicated(subset=['input']).sum()
print(f"Jumlah duplikasi pada kolom 'input': {duplicates_input}")

# Tampilkan contoh data duplikat jika ada
if duplicates_input > 0:
    print("\nContoh data duplikat (berdasarkan kolom 'input'):")
    duplicate_samples = df_clean[df_clean.duplicated(subset=['input'], keep=False)].sort_values('input').head(10)
    display(duplicate_samples[['input', 'output']])

In [ ]:
# Menghapus duplikasi
initial_rows = len(df_clean)

# Hapus duplikat berdasarkan kolom 'input', keep='first' untuk mempertahankan kemunculan pertama
df_clean = df_clean.drop_duplicates(subset=['input'], keep='first')

removed_duplicates = initial_rows - len(df_clean)

print(f"Jumlah baris duplikat yang dihapus: {removed_duplicates}")
print(f"Total data setelah menghapus duplikasi: {len(df_clean)} baris")

# Reset index setelah menghapus duplikasi
df_clean = df_clean.reset_index(drop=True)
print(f"Index berhasil direset.")

## 6. Text Cleaning dan Preprocessing

Tahap ini melakukan pembersihan teks untuk memastikan data siap diproses oleh model. Proses ini menghapus noise seperti URL, HTML tags, karakter khusus, dan menormalisasi whitespace.

In [ ]:
def advanced_text_cleaning(text):
    """
    Fungsi untuk membersihkan teks dari berbagai noise dan inkonsistensi.

    Parameters:
    -----------
    text : str
        Teks yang akan dibersihkan

    Returns:
    --------
    str
        Teks yang sudah dibersihkan

    Proses cleaning meliputi:
    1. Menghapus URL (http, https, www)
    2. Menghapus HTML tags
    3. Menghapus karakter khusus (hanya menyisakan huruf, angka, dan spasi)
    4. Normalisasi whitespace (mengganti multiple spaces dengan single space)
    """

    # Handle input yang bukan string
    if not isinstance(text, str):
        return str(text) if text is not None else ""

    # Hapus URL dengan pattern matching
    # Pattern ini menangkap URL yang dimulai dengan http://, https://, atau www.
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Hapus HTML tags menggunakan regex
    # Pattern <.*?> menangkap semua tag HTML
    text = re.sub(r'<.*?>', '', text)

    # Hapus karakter khusus, hanya pertahankan huruf (a-z, A-Z), angka (0-9), dan spasi
    # Ini membantu mengurangi dimensionalitas dan noise dalam data
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Normalisasi whitespace: ganti multiple spaces dengan single space
    # dan hapus leading/trailing spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text


# Aplikasikan fungsi cleaning ke kolom 'input'
print("Memulai proses text cleaning...")
df_clean['cleaned_input'] = df_clean['input'].apply(advanced_text_cleaning)

# Tampilkan perbandingan sebelum dan sesudah cleaning
print("\n" + "=" * 80)
print("PERBANDINGAN TEKS SEBELUM DAN SESUDAH CLEANING")
print("=" * 80)

comparison_df = df_clean[['input', 'cleaned_input']].head()
for idx, row in comparison_df.iterrows():
    print(f"\n--- Sample {idx + 1} ---")
    print(f"Original  : {row['input'][:150]}...")
    print(f"Cleaned   : {row['cleaned_input'][:150]}...")

print(f"\nText cleaning selesai untuk {len(df_clean)} baris data.")

## 7. Ekstraksi dan Transformasi Label Sentimen

Pada tahap ini, kita mengekstrak label sentimen dari kolom output dan melakukan normalisasi kategori sentimen (menggabungkan 'very positive' dengan 'positive' dan 'very negative' dengan 'negative').

In [ ]:
# Ekstraksi label sentimen dari kolom output
# Format awal: "A: very positive", "B: positive", dst.
# Kita hapus prefix (A:, B:, dst) untuk mendapatkan label yang bersih

df_clean['sentiment'] = df_clean['output'].str.replace(r'^[A-E]:\s*', '', regex=True)

print("=" * 80)
print("DISTRIBUSI SENTIMEN SEBELUM NORMALISASI")
print("=" * 80)
print(df_clean['sentiment'].value_counts())

# Normalisasi label sentimen
# Menggabungkan kategori yang sangat mirip untuk menyederhanakan klasifikasi
# very positive -> positive, very negative -> negative
sentiment_normalization = {
    'very positive': 'positive',
    'very negative': 'negative',
    'positive': 'positive',
    'negative': 'negative',
    'neutral': 'neutral'
}

df_clean['sentiment'] = df_clean['sentiment'].map(sentiment_normalization)

print("\n" + "=" * 80)
print("DISTRIBUSI SENTIMEN SETELAH NORMALISASI")
print("=" * 80)
print(df_clean['sentiment'].value_counts())
print("\nProporsi masing-masing kelas:")
print(df_clean['sentiment'].value_counts(normalize=True))

# Tampilkan sample hasil ekstraksi
print("\n" + "=" * 80)
print("SAMPLE HASIL EKSTRAKSI SENTIMEN")
print("=" * 80)
display(df_clean[['output', 'sentiment', 'cleaned_input']].head(10))

## 8. Visualisasi Distribusi Sentimen

Visualisasi membantu kita memahami balance/imbalance dari kelas sentimen dalam dataset. Ini penting untuk menentukan apakah perlu dilakukan teknik balancing seperti oversampling atau undersampling.

In [ ]:
# Konfigurasi warna untuk setiap sentimen agar visualisasi lebih informatif
sentiment_colors = {
    'positive': '#2ecc71',  # Hijau untuk sentimen positif
    'negative': '#e74c3c',  # Merah untuk sentimen negatif
    'neutral': '#3498db'    # Biru untuk sentimen netral
}

# Membuat visualisasi distribusi sentimen
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Count plot
sns.countplot(
    x='sentiment',
    data=df_clean,
    hue='sentiment',
    palette=sentiment_colors,
    legend=False,
    ax=axes[0]
)
axes[0].set_title('Distribusi Sentimen (Count)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Kategori Sentimen', fontsize=12)
axes[0].set_ylabel('Jumlah Data', fontsize=12)

# Tambahkan nilai di atas setiap bar
for container in axes[0].containers:
    axes[0].bar_label(container, fmt='%d')

# Plot 2: Pie chart untuk proporsi
sentiment_counts = df_clean['sentiment'].value_counts()
colors = [sentiment_colors[label] for label in sentiment_counts.index]

axes[1].pie(
    sentiment_counts.values,
    labels=sentiment_counts.index,
    autopct='%1.1f%%',
    colors=colors,
    startangle=90,
    textprops={'fontsize': 11}
)
axes[1].set_title('Proporsi Sentimen', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Analisis class imbalance
print("\n" + "=" * 80)
print("ANALISIS CLASS IMBALANCE")
print("=" * 80)
majority_class = sentiment_counts.idxmax()
minority_class = sentiment_counts.idxmin()
imbalance_ratio = sentiment_counts.max() / sentiment_counts.min()

print(f"Kelas mayoritas: {majority_class} ({sentiment_counts.max()} data)")
print(f"Kelas minoritas: {minority_class} ({sentiment_counts.min()} data)")
print(f"Rasio imbalance: {imbalance_ratio:.2f}:1")

if imbalance_ratio > 3:
    print("\n⚠️  PERINGATAN: Dataset mengalami class imbalance yang signifikan!")
    print("   Rekomendasi: Gunakan teknik oversampling atau class weighting saat training model.")
else:
    print("\n✓ Dataset relatif seimbang.")

## 9. Encoding Label Kategorikal

Machine learning model memerlukan input numerik. Oleh karena itu, kita perlu mengubah label sentimen kategorikal menjadi representasi numerik menggunakan label encoding.

In [ ]:
# Definisi mapping untuk encoding sentimen
# Mapping ini konsisten dan akan digunakan sepanjang proses
sentiment_mapping = {
    'positive': 0,
    'neutral': 1,
    'negative': 2
}

# Aplikasikan encoding
df_clean['sentiment_encoded'] = df_clean['sentiment'].map(sentiment_mapping)

print("=" * 80)
print("SENTIMENT ENCODING MAPPING")
print("=" * 80)
for sentiment, code in sentiment_mapping.items():
    count = (df_clean['sentiment_encoded'] == code).sum()
    print(f"{sentiment:10s} -> {code} ({count:6d} data)")

print("\n" + "=" * 80)
print("SAMPLE DATA DENGAN ENCODED SENTIMEN")
print("=" * 80)
display(df_clean[['cleaned_input', 'sentiment', 'sentiment_encoded']].head(10))

# Verifikasi tidak ada missing values setelah encoding
assert df_clean['sentiment_encoded'].isnull().sum() == 0, "Terdapat nilai yang tidak termap dalam encoding!"
print("\n✓ Encoding berhasil, tidak ada nilai yang terlewat.")

## 10. Analisis Panjang Teks

Sebelum split data, kita perlu memahami distribusi panjang teks. Ini membantu dalam menentukan parameter seperti max_length untuk tokenization.

In [ ]:
# Hitung panjang teks (jumlah karakter dan jumlah kata)
df_clean['text_length_chars'] = df_clean['cleaned_input'].str.len()
df_clean['text_length_words'] = df_clean['cleaned_input'].str.split().str.len()

print("=" * 80)
print("STATISTIK PANJANG TEKS")
print("=" * 80)
print("\nBerdasarkan jumlah karakter:")
print(df_clean['text_length_chars'].describe())

print("\nBerdasarkan jumlah kata:")
print(df_clean['text_length_words'].describe())

# Visualisasi distribusi panjang teks
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram panjang karakter
axes[0].hist(df_clean['text_length_chars'], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].axvline(df_clean['text_length_chars'].median(), color='red', linestyle='--',
                label=f"Median: {df_clean['text_length_chars'].median():.0f}")
axes[0].set_xlabel('Jumlah Karakter', fontsize=12)
axes[0].set_ylabel('Frekuensi', fontsize=12)
axes[0].set_title('Distribusi Panjang Teks (Karakter)', fontsize=14, fontweight='bold')
axes[0].legend()

# Histogram panjang kata
axes[1].hist(df_clean['text_length_words'], bins=50, color='lightcoral', edgecolor='black', alpha=0.7)
axes[1].axvline(df_clean['text_length_words'].median(), color='blue', linestyle='--',
                label=f"Median: {df_clean['text_length_words'].median():.0f}")
axes[1].set_xlabel('Jumlah Kata', fontsize=12)
axes[1].set_ylabel('Frekuensi', fontsize=12)
axes[1].set_title('Distribusi Panjang Teks (Kata)', fontsize=14, fontweight='bold')
axes[1].legend()

plt.tight_layout()
plt.show()

## 11. Split Data: Training, Validation, dan Testing

Data split dilakukan dengan proporsi 70% training, 15% validation, dan 15% testing. Stratifikasi digunakan untuk memastikan distribusi sentimen yang konsisten di setiap set.

In [ ]:
# Konstanta untuk reproducibility
RANDOM_STATE = 42
TEST_SIZE = 0.3  # 30% untuk gabungan validation + testing
VAL_TEST_SPLIT = 0.5  # 50% dari 30% = 15% untuk masing-masing validation dan testing

print("=" * 80)
print("PROSES SPLIT DATA")
print("=" * 80)

# Step 1: Split menjadi training (70%) dan temporary set (30%)
df_train, df_temp, _, _ = train_test_split(
    df_clean,
    df_clean['sentiment_encoded'],
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=df_clean['sentiment_encoded']
)

print(f"Step 1: Split data menjadi training (70%) dan temporary (30%)")
print(f"  - Training set: {len(df_train)} baris")
print(f"  - Temporary set: {len(df_temp)} baris")

# Step 2: Split temporary set menjadi validation (15%) dan testing (15%)
# test_size=0.5 karena 0.5 * 30% = 15% dari dataset original
df_val, df_test, _, _ = train_test_split(
    df_temp,
    df_temp['sentiment_encoded'],
    test_size=VAL_TEST_SPLIT,
    random_state=RANDOM_STATE,
    stratify=df_temp['sentiment_encoded']
)

print(f"\nStep 2: Split temporary set menjadi validation (15%) dan testing (15%)")
print(f"  - Validation set: {len(df_val)} baris")
print(f"  - Testing set: {len(df_test)} baris")

# Verifikasi proporsi
print("\n" + "=" * 80)
print("VERIFIKASI PROPORSI SPLIT")
print("=" * 80)
total_data = len(df_clean)
print(f"Total data original: {total_data}")
print(f"Training   : {len(df_train):6d} baris ({len(df_train)/total_data*100:.1f}%)")
print(f"Validation : {len(df_val):6d} baris ({len(df_val)/total_data*100:.1f}%)")
print(f"Testing    : {len(df_test):6d} baris ({len(df_test)/total_data*100:.1f}%)")
print(f"Total      : {len(df_train) + len(df_val) + len(df_test):6d} baris")

# Verifikasi stratifikasi - distribusi sentimen harus konsisten
print("\n" + "=" * 80)
print("VERIFIKASI STRATIFIKASI - DISTRIBUSI SENTIMEN")
print("=" * 80)

print("\nTraining Set:")
print(df_train['sentiment'].value_counts(normalize=True).sort_index())

print("\nValidation Set:")
print(df_val['sentiment'].value_counts(normalize=True).sort_index())

print("\nTesting Set:")
print(df_test['sentiment'].value_counts(normalize=True).sort_index())

print("\n✓ Stratifikasi berhasil - distribusi sentimen konsisten di setiap set.")

## 12. Oversampling pada Training Set

Class imbalance dapat menyebabkan model bias terhadap kelas mayoritas. Untuk mengatasi ini, kita lakukan oversampling dengan teknik random sampling dengan replacement pada kelas minoritas.

**Catatan Penting**: Oversampling hanya dilakukan pada training set untuk mencegah data leakage. Validation dan testing set dibiarkan natural untuk evaluasi yang fair.

In [ ]:
print("=" * 80)
print("OVERSAMPLING TRAINING SET")
print("=" * 80)

# Identifikasi kelas mayoritas dan jumlahnya
majority_count = df_train['sentiment'].value_counts().max()
majority_class = df_train['sentiment'].value_counts().idxmax()

print(f"Kelas mayoritas: {majority_class}")
print(f"Target jumlah per kelas: {majority_count} samples\n")

print("Distribusi SEBELUM oversampling:")
print(df_train['sentiment'].value_counts().sort_index())

# Inisialisasi dataframe untuk hasil oversampling
df_train_oversampled = pd.DataFrame()

# Lakukan oversampling untuk setiap kelas sentimen
for sentiment_label in df_train['sentiment'].unique():
    # Ambil data untuk sentimen tertentu
    sentiment_df = df_train[df_train['sentiment'] == sentiment_label]
    current_count = len(sentiment_df)

    if current_count < majority_count:
        # Kelas minoritas: lakukan oversampling dengan random sampling (replacement=True)
        # Ini akan menduplikasi beberapa sample secara random hingga mencapai jumlah mayoritas
        oversampled_df = sentiment_df.sample(
            n=majority_count,
            replace=True,
            random_state=RANDOM_STATE
        )
        print(f"  {sentiment_label:10s}: {current_count:5d} -> {len(oversampled_df):5d} samples (oversampled)")
    else:
        # Kelas mayoritas: gunakan semua data tanpa perubahan
        oversampled_df = sentiment_df
        print(f"  {sentiment_label:10s}: {current_count:5d} samples (unchanged)")

    # Gabungkan ke dataframe hasil
    df_train_oversampled = pd.concat([df_train_oversampled, oversampled_df], ignore_index=True)

# Shuffle data setelah oversampling untuk menghindari ordering bias
df_train_oversampled = df_train_oversampled.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

print("\nDistribusi SETELAH oversampling:")
print(df_train_oversampled['sentiment'].value_counts().sort_index())

print("\n" + "=" * 80)
print("SUMMARY OVERSAMPLING")
print("=" * 80)
print(f"Ukuran training set sebelum oversampling: {len(df_train):6d} baris")
print(f"Ukuran training set setelah oversampling: {len(df_train_oversampled):6d} baris")
print(f"Peningkatan: {len(df_train_oversampled) - len(df_train):6d} baris ({(len(df_train_oversampled)/len(df_train) - 1)*100:.1f}%)")

print("\n✓ Oversampling selesai - semua kelas memiliki jumlah sample yang seimbang.")

## 13. Visualisasi Perbandingan Sebelum dan Sesudah Oversampling

In [ ]:
# Membuat visualisasi perbandingan
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot sebelum oversampling
df_train['sentiment'].value_counts().sort_index().plot(
    kind='bar',
    ax=axes[0],
    color=['#2ecc71', '#e74c3c', '#3498db'],
    edgecolor='black',
    alpha=0.7
)
axes[0].set_title('Distribusi Sentimen SEBELUM Oversampling', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Kategori Sentimen', fontsize=12)
axes[0].set_ylabel('Jumlah Data', fontsize=12)
axes[0].set_xticklabels(['Negative', 'Neutral', 'Positive'], rotation=0)

# Tambahkan nilai di atas bar
for container in axes[0].containers:
    axes[0].bar_label(container, fmt='%d')

# Plot setelah oversampling
df_train_oversampled['sentiment'].value_counts().sort_index().plot(
    kind='bar',
    ax=axes[1],
    color=['#2ecc71', '#e74c3c', '#3498db'],
    edgecolor='black',
    alpha=0.7
)
axes[1].set_title('Distribusi Sentimen SETELAH Oversampling', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Kategori Sentimen', fontsize=12)
axes[1].set_ylabel('Jumlah Data', fontsize=12)
axes[1].set_xticklabels(['Negative', 'Neutral', 'Positive'], rotation=0)

# Tambahkan nilai di atas bar
for container in axes[1].containers:
    axes[1].bar_label(container, fmt='%d')

plt.tight_layout()
plt.show()

## 14. Tokenisasi dengan BERT Tokenizer

Untuk model berbasis transformer seperti BERT, kita perlu mengubah teks menjadi token numerik. Proses ini menggunakan pre-trained BERT tokenizer yang sudah dilatih pada vocabulary yang luas.

In [ ]:
# Load pre-trained BERT tokenizer
# bert-base-uncased: versi BERT dengan lowercase text
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

print("=" * 80)
print("BERT TOKENIZER")
print("=" * 80)
print(f"Model tokenizer: bert-base-uncased")
print(f"Vocabulary size: {tokenizer.vocab_size:,} tokens")
print(f"Model max length: {tokenizer.model_max_length} tokens")
print("\n✓ BERT Tokenizer berhasil dimuat.")

## 15. Analisis Distribusi Panjang Token

Sebelum menentukan max_length untuk tokenization, kita perlu menganalisis distribusi panjang token dalam data. Ini membantu kita menemukan nilai optimal yang bisa menangani sebagian besar data tanpa terlalu banyak padding.

In [ ]:
# Hitung panjang token untuk setiap teks dalam training set yang sudah di-oversample
# Kita gunakan truncation di sini hanya untuk analisis, bukan untuk tokenization final
print("Menghitung panjang token untuk analisis distribusi...")

token_lengths = [
    len(tokenizer.encode(
        str(text),
        add_special_tokens=True,
        truncation=True,
        max_length=tokenizer.model_max_length
    ))
    for text in df_train_oversampled['cleaned_input']
]

print(f"Selesai menghitung {len(token_lengths)} samples.\n")

# Statistik deskriptif
print("=" * 80)
print("STATISTIK PANJANG TOKEN")
print("=" * 80)
print(f"Minimum      : {np.min(token_lengths)} tokens")
print(f"Maksimum     : {np.max(token_lengths)} tokens")
print(f"Mean         : {np.mean(token_lengths):.2f} tokens")
print(f"Median       : {np.median(token_lengths):.0f} tokens")
print(f"Std Dev      : {np.std(token_lengths):.2f} tokens")
print(f"Percentile 90: {np.percentile(token_lengths, 90):.0f} tokens")
print(f"Percentile 95: {np.percentile(token_lengths, 95):.0f} tokens")
print(f"Percentile 99: {np.percentile(token_lengths, 99):.0f} tokens")

# Visualisasi distribusi
plt.figure(figsize=(12, 6))
sns.histplot(token_lengths, bins=50, kde=True, color='steelblue', edgecolor='black', alpha=0.6)

# Tambahkan garis vertikal untuk statistik penting
plt.axvline(np.mean(token_lengths), color='red', linestyle='--',
            label=f'Mean: {np.mean(token_lengths):.0f}', linewidth=2)
plt.axvline(np.percentile(token_lengths, 95), color='orange', linestyle='--',
            label=f'95th Percentile: {np.percentile(token_lengths, 95):.0f}', linewidth=2)
plt.axvline(np.percentile(token_lengths, 99), color='green', linestyle='--',
            label=f'99th Percentile: {np.percentile(token_lengths, 99):.0f}', linewidth=2)

plt.xlabel('Jumlah Token', fontsize=12)
plt.ylabel('Frekuensi', fontsize=12)
plt.title('Distribusi Panjang Token pada Training Set (Oversampled)',
          fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Tentukan MAX_LENGTH optimal
# Kita gunakan percentile 95 atau 99 untuk coverage yang baik tanpa terlalu banyak padding
# Namun tetap dibatasi maksimal 512 (limit BERT)
suggested_max_length = int(np.percentile(token_lengths, 95))
MAX_LENGTH = min(suggested_max_length, 512)

# Untuk efisiensi, kita bisa set ke nilai yang lebih reasonable jika terlalu panjang
if MAX_LENGTH > 400:
    MAX_LENGTH = 330  # Nilai praktis yang masih cover majority data

print("\n" + "=" * 80)
print("PENENTUAN MAX_LENGTH")
print("=" * 80)
print(f"Suggested max_length (95th percentile): {suggested_max_length} tokens")
print(f"Chosen MAX_LENGTH: {MAX_LENGTH} tokens")
print(f"Coverage: ~{(np.array(token_lengths) <= MAX_LENGTH).sum() / len(token_lengths) * 100:.1f}% dari data")

## 16. Tokenisasi Final pada Semua Dataset Split

Dengan MAX_LENGTH yang sudah ditentukan, kita lakukan tokenisasi pada training, validation, dan testing set. Proses ini mengkonversi teks menjadi format yang siap digunakan oleh model BERT.

In [ ]:
def tokenize_data(texts, tokenizer, max_length):
    """
    Fungsi untuk melakukan tokenisasi pada sekumpulan teks.

    Parameters:
    -----------
    texts : Series/List
        Koleksi teks yang akan ditokenisasi
    tokenizer : PreTrainedTokenizer
        BERT tokenizer
    max_length : int
        Panjang maksimal sequence setelah tokenization

    Returns:
    --------
    BatchEncoding
        Dictionary berisi input_ids, attention_mask, dan token_type_ids dalam format PyTorch tensor

    Process:
    --------
    - padding='max_length': Tambahkan padding hingga max_length
    - truncation=True: Potong teks yang melebihi max_length
    - return_tensors='pt': Return dalam format PyTorch tensor
    """
    return tokenizer(
        list(texts),
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )


print("=" * 80)
print("TOKENISASI DATASET")
print("=" * 80)

# Tokenisasi training set (yang sudah di-oversample)
print("Melakukan tokenisasi pada training set (oversampled)...")
X_train_tokenized = tokenize_data(df_train_oversampled['cleaned_input'], tokenizer, MAX_LENGTH)
print(f"  ✓ Selesai - Shape: {X_train_tokenized['input_ids'].shape}")

# Tokenisasi validation set
print("Melakukan tokenisasi pada validation set...")
X_val_tokenized = tokenize_data(df_val['cleaned_input'], tokenizer, MAX_LENGTH)
print(f"  ✓ Selesai - Shape: {X_val_tokenized['input_ids'].shape}")

# Tokenisasi testing set
print("Melakukan tokenisasi pada testing set...")
X_test_tokenized = tokenize_data(df_test['cleaned_input'], tokenizer, MAX_LENGTH)
print(f"  ✓ Selesai - Shape: {X_test_tokenized['input_ids'].shape}")

print("\n" + "=" * 80)
print("SUMMARY TOKENISASI")
print("=" * 80)
print(f"MAX_LENGTH yang digunakan: {MAX_LENGTH} tokens")
print(f"\nTraining set   : {X_train_tokenized['input_ids'].shape[0]:6d} samples × {MAX_LENGTH} tokens")
print(f"Validation set : {X_val_tokenized['input_ids'].shape[0]:6d} samples × {MAX_LENGTH} tokens")
print(f"Testing set    : {X_test_tokenized['input_ids'].shape[0]:6d} samples × {MAX_LENGTH} tokens")

print("\n✓ Tokenisasi berhasil untuk semua dataset split.")

## 17. Konversi ke PyTorch Tensors

Model deep learning memerlukan input dalam format tensor. Kita konversi hasil tokenisasi dan label menjadi PyTorch tensors untuk keperluan training.

In [ ]:
print("=" * 80)
print("KONVERSI KE PYTORCH TENSORS")
print("=" * 80)

# Training set tensors
print("Memproses training set...")
input_ids_train = X_train_tokenized['input_ids']
attention_mask_train = X_train_tokenized['attention_mask']
token_type_ids_train = X_train_tokenized['token_type_ids']
labels_train = torch.tensor(df_train_oversampled['sentiment_encoded'].values.astype(int))
print(f"  ✓ Labels shape: {labels_train.shape}")

# Validation set tensors
print("Memproses validation set...")
input_ids_val = X_val_tokenized['input_ids']
attention_mask_val = X_val_tokenized['attention_mask']
token_type_ids_val = X_val_tokenized['token_type_ids']
labels_val = torch.tensor(df_val['sentiment_encoded'].values.astype(int))
print(f"  ✓ Labels shape: {labels_val.shape}")

# Testing set tensors
print("Memproses testing set...")
input_ids_test = X_test_tokenized['input_ids']
attention_mask_test = X_test_tokenized['attention_mask']
token_type_ids_test = X_test_tokenized['token_type_ids']
labels_test = torch.tensor(df_test['sentiment_encoded'].values.astype(int))
print(f"  ✓ Labels shape: {labels_test.shape}")

print("\n✓ Semua data berhasil dikonversi ke PyTorch tensors.")

## 18. Pembuatan TensorDataset dan DataLoader

TensorDataset menggabungkan input tensors dan labels, sedangkan DataLoader menangani batching dan shuffling untuk proses training yang efisien.

In [ ]:
# Buat TensorDataset untuk setiap split
print("=" * 80)
print("PEMBUATAN TENSORDATASET")
print("=" * 80)

train_dataset = TensorDataset(
    input_ids_train,
    attention_mask_train,
    token_type_ids_train,
    labels_train
)
print(f"Training dataset   : {len(train_dataset)} samples")

val_dataset = TensorDataset(
    input_ids_val,
    attention_mask_val,
    token_type_ids_val,
    labels_val
)
print(f"Validation dataset : {len(val_dataset)} samples")

test_dataset = TensorDataset(
    input_ids_test,
    attention_mask_test,
    token_type_ids_test,
    labels_test
)
print(f"Testing dataset    : {len(test_dataset)} samples")

# Buat DataLoader dengan batch size yang optimal
BATCH_SIZE = 32

print("\n" + "=" * 80)
print("PEMBUATAN DATALOADER")
print("=" * 80)
print(f"Batch size: {BATCH_SIZE}\n")

train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True  # Shuffle untuk training agar model tidak belajar dari urutan data
)
print(f"Training DataLoader   : {len(train_dataloader)} batches")

val_dataloader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False  # Tidak perlu shuffle untuk validation
)
print(f"Validation DataLoader : {len(val_dataloader)} batches")

test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False  # Tidak perlu shuffle untuk testing
)
print(f"Testing DataLoader    : {len(test_dataloader)} batches")

print("\n✓ TensorDataset dan DataLoader berhasil dibuat.")

## 19. Data Dictionary

Data dictionary menjelaskan setiap variabel/kolom dalam dataset yang sudah diproses, termasuk tipe data, deskripsi, dan contoh nilai.

In [ ]:
# Membuat data dictionary
data_dictionary = {
    'Nama Kolom': [
        'input',
        'output',
        'cleaned_input',
        'sentiment',
        'sentiment_encoded',
        'text_length_chars',
        'text_length_words'
    ],
    'Tipe Data': [
        'object (string)',
        'object (string)',
        'object (string)',
        'object (string)',
        'int64',
        'int64',
        'int64'
    ],
    'Deskripsi': [
        'Teks input original dari dataset sebelum preprocessing',
        'Label output original dengan format prefix (A:, B:, dst) dan kategori sentimen',
        'Teks input yang sudah dibersihkan dari URL, HTML tags, karakter khusus, dan whitespace yang tidak perlu',
        'Label sentimen kategorikal hasil ekstraksi dan normalisasi (positive, negative, neutral)',
        'Label sentimen dalam format numerik untuk keperluan machine learning (0: positive, 1: neutral, 2: negative)',
        'Jumlah karakter dalam teks yang sudah dibersihkan',
        'Jumlah kata dalam teks yang sudah dibersihkan'
    ],
    'Contoh Nilai': [
        'Check out this amazing product! http://example.com',
        'A: very positive',
        'Check out this amazing product',
        'positive',
        '0',
        '34',
        '5'
    ],
    'Missing Values': [
        '0',
        '0',
        '0',
        '0',
        '0',
        '0',
        '0'
    ]
}

df_dictionary = pd.DataFrame(data_dictionary)

print("=" * 80)
print("DATA DICTIONARY - DATASET PREPROCESSING")
print("=" * 80)
display(df_dictionary)

# Informasi tambahan tentang encoding
print("\n" + "=" * 80)
print("SENTIMENT ENCODING REFERENCE")
print("=" * 80)
for sentiment, code in sentiment_mapping.items():
    print(f"{code} → {sentiment}")

# Informasi tentang transformasi yang dilakukan
print("\n" + "=" * 80)
print("TRANSFORMASI DATA YANG DILAKUKAN")
print("=" * 80)
transformations = [
    "1. Cleaning: Menghapus duplikasi data berdasarkan kolom 'input'",
    "2. Handling Missing Values: Menghapus baris dengan missing values pada kolom 'input' dan 'output'",
    "3. Text Cleaning: Menghapus URL, HTML tags, karakter khusus, dan normalisasi whitespace",
    "4. Label Extraction: Ekstraksi label sentimen dari kolom 'output'",
    "5. Label Normalization: Menggabungkan 'very positive' dengan 'positive' dan 'very negative' dengan 'negative'",
    "6. Categorical Encoding: Mengubah label sentimen kategorikal menjadi numerik",
    "7. Data Split: Membagi data menjadi training (70%), validation (15%), testing (15%) dengan stratifikasi",
    "8. Oversampling: Menyeimbangkan kelas pada training set menggunakan random oversampling",
    "9. Tokenization: Mengkonversi teks menjadi token numerik menggunakan BERT tokenizer",
    "10. Tensor Conversion: Mengkonversi data menjadi PyTorch tensors untuk model training"
]

for transformation in transformations:
    print(f"  {transformation}")

## 20. Summary dan Hasil Akhir Preprocessing

Ringkasan lengkap dari semua proses preprocessing yang telah dilakukan, termasuk statistik dataset dan output yang dihasilkan.

In [ ]:
print("=" * 80)
print("LAPORAN AKHIR DATA WRANGLING DAN PREPROCESSING")
print("=" * 80)

print("\n📊 STATISTIK DATASET")
print("-" * 80)
print(f"Dataset original          : {len(df):,} baris")
print(f"Baris duplikat dihapus    : {removed_duplicates:,} baris")
print(f"Dataset setelah cleaning  : {len(df_clean):,} baris")
print(f"Missing values ditangani  : Ya (drop baris dengan missing values pada kolom penting)")

print("\n📂 DATA SPLIT")
print("-" * 80)
print(f"Training set (original)   : {len(df_train):,} baris (70%)")
print(f"Training set (oversampled): {len(df_train_oversampled):,} baris")
print(f"Validation set            : {len(df_val):,} baris (15%)")
print(f"Testing set               : {len(df_test):,} baris (15%)")

print("\n🔄 TRANSFORMASI")
print("-" * 80)
print(f"Text cleaning             : ✓ Selesai")
print(f"Label encoding            : ✓ Selesai (3 kelas)")
print(f"Oversampling              : ✓ Selesai (balance ratio 1:1:1)")
print(f"Tokenization              : ✓ Selesai (BERT, max_length={MAX_LENGTH})")
print(f"Tensor conversion         : ✓ Selesai (PyTorch tensors)")

print("\n📈 DISTRIBUSI SENTIMEN (Training Set Oversampled)")
print("-" * 80)
for sentiment in ['negative', 'neutral', 'positive']:
    count = (df_train_oversampled['sentiment'] == sentiment).sum()
    percentage = count / len(df_train_oversampled) * 100
    print(f"{sentiment.capitalize():10s}: {count:6,} samples ({percentage:5.1f}%)")

print("\n✅ OUTPUT YANG DIHASILKAN")
print("-" * 80)
outputs = [
    "1. Dataset bersih (df_clean): Dataset yang sudah melalui cleaning, handling missing values, dan transformasi",
    "2. Training set (df_train_oversampled): Data training yang sudah di-balance dengan oversampling",
    "3. Validation set (df_val): Data untuk validasi model selama training",
    "4. Testing set (df_test): Data untuk evaluasi performa final model",
    "5. PyTorch DataLoaders: train_dataloader, val_dataloader, test_dataloader siap untuk model training",
    "6. Data Dictionary: Dokumentasi lengkap setiap variabel dalam dataset",
    "7. Laporan Preprocessing: Dokumentasi semua tahapan dan hasil preprocessing"
]

for output in outputs:
    print(f"  {output}")

print("\n" + "=" * 80)
print("PREPROCESSING SELESAI - DATASET SIAP UNTUK MODEL TRAINING")
print("=" * 80)

## 21. Ekspor Dataset Bersih (Opsional)

Menyimpan dataset yang sudah dibersihkan untuk keperluan dokumentasi atau analisis lebih lanjut.

In [ ]:
# Simpan dataset bersih ke CSV (opsional)
output_filename = 'sentiment_dataset_cleaned.csv'

# Pilih kolom yang relevan untuk disimpan
columns_to_save = [
    'input',
    'output',
    'cleaned_input',
    'sentiment',
    'sentiment_encoded',
    'text_length_chars',
    'text_length_words'
]

df_clean[columns_to_save].to_csv(output_filename, index=False)

print(f"✓ Dataset bersih berhasil disimpan ke: {output_filename}")
print(f"  Total baris: {len(df_clean):,}")
print(f"  Total kolom: {len(columns_to_save)}")

---

## Kesimpulan

Proses data wrangling dan preprocessing telah berhasil diselesaikan dengan tahapan sebagai berikut:

### 1. **Data Cleaning**
   - Menghapus duplikasi data untuk menghindari bias
   - Menangani missing values dengan strategi drop pada kolom penting
   - Membersihkan teks dari noise (URL, HTML tags, karakter khusus)
   - Normalisasi whitespace untuk konsistensi

### 2. **Penanganan Missing Values**
   - Identifikasi missing values pada semua kolom
   - Penghapusan baris dengan missing values pada kolom 'input' dan 'output'
   - Verifikasi tidak ada missing values setelah cleaning

### 3. **Transformasi Data**
   - **Encoding kategorikal**: Label sentimen diubah menjadi numerik (positive: 0, neutral: 1, negative: 2)
   - **Normalisasi label**: Menggabungkan kategori sentimen yang mirip (very positive → positive, very negative → negative)
   - **Oversampling**: Menyeimbangkan distribusi kelas pada training set menggunakan random oversampling
   - **Tokenization**: Mengkonversi teks menjadi format yang sesuai untuk model BERT

### 4. **Data Dictionary**
   - Dokumentasi lengkap untuk setiap variabel dalam dataset
   - Penjelasan tipe data, deskripsi, dan contoh nilai
   - Referensi encoding untuk label sentimen

### Output Akhir:
- ✅ Dataset bersih dan siap digunakan
- ✅ Data split dengan proporsi optimal (70-15-15)
- ✅ Training set yang balanced untuk menghindari bias model
- ✅ PyTorch DataLoaders siap untuk training model BERT
- ✅ Dokumentasi lengkap proses preprocessing

Dataset sekarang siap untuk tahap selanjutnya: **Model Training dan Evaluation**.

---

**Catatan**: Semua transformasi dilakukan dengan mempertahankan integritas data dan mencegah data leakage antara training, validation, dan testing set.